pip3 install scipy==1.13.1 && \
pip3 install -U scikit-learn && \
pip3 install scanpy==1.10.3 && \
pip3 install anndata==0.10.5.post1 && \
pip3 install einops==0.8.0 && \
pip3 install local-attention==1.9.14 && \
pip3 install gdown && \
pip3 install memory-profiler && \
pip3 installmega.py


In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os
import gc
import argparse
import json
import random
import math
import functools												 
import numpy as np 
import pandas as pd
from collections import Counter
from pathlib import Path

import scipy    
import sklearn 
import scanpy as sc
import anndata as ad

import utils as utils

In [9]:
data_dir = Path('performer_pytorch')
data_dir.mkdir(exist_ok=True)  

from performer_pytorch.performer_pytorch import PerformerLM

In [10]:
filename = 'panglao_human.h5ad'
data_dir = Path('data')
data_dir.mkdir(exist_ok=True)  
path_data = data_dir / filename

if False: 
    !gdown https://drive.google.com/uc?id=15biFWDLipYpM9iTGgaPmBhlgjkUu08bz -O {path_data} 

if False: 
    from mega import Mega
    mega = Mega()
    m = mega.login()
    file = m.download_url('https://mega.nz/file/thZVlZrC#g7WYLCIXyBSdabju3jpfLUanx_MTaB4rmhV3h1RnFcU', 
                          dest_filename=path_data)

In [11]:
path_gne2vec = data_dir / 'gene2vec_16906.npy'

if False: 
    !gdown https://drive.google.com/uc?id=15aCnM4kyTBwe6S39RsqJ8JJnE9Xj-gyi -O {path_gne2vec}

In [12]:
data = sc.read_h5ad(path_data)

print(type(data))
print(data.shape)

# Each row corresponds to a cell's gene expression data, and each column corresponds to a specific gene.
pd.DataFrame(data[:10, :17].X.toarray(), columns=data[:10, :17].var_names, index=data[:10, :17].obs_names)         

<class 'anndata._core.anndata.AnnData'>
(1357593, 16906)


/usr/local/lib/python3.10/site-packages/anndata/_core/anndata.py:1906: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


,A1BG,A1CF,A2ML1,A2M,A4GALT,A4GNT,AAAS,AACS,AADACL2,AADAC,AADAT,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2
09cbFwH3,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1.840998,0.00000,0.000000,0.0,0.000000
0AdkBIuA,0.0,0.0,0.0,0.000000,0.0,0.0,0.028411,0.028411,0.0,0.0,0.0,0.0,1.626798,0.00000,0.000000,0.0,0.000000
0frx9XJU,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.049059,0.0,0.000000
0NaEBg2H,0.0,0.0,0.0,0.453423,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.037561,0.00000,0.000000,0.0,3.388124
0NQlCyEz,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.750278,0.00000,0.016584,0.0,0.000000
0WaPBWQw,0.0,0.0,0.0,0.000000,0.0,0.0,2.452006,2.610123,0.0,0.0,0.0,0.0,0.012644,0.00000,1.919015,0.0,0.805893
12cCwzii,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1.166541,0.00000,0.000000,0.0,0.000000
1iEKkCnT,0.0,0.0,0.0,0.515515,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,3.563018,0.00000,0.000000,0.0,0.000000
1lIceRcL,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,1.39369,0.000000,0.0,2.837606
1MhXbC7E,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1.961442,0.00000,0.000000,0.0,0.000000


In [13]:
print(data.shape)
len(set(data.obs_names)), len(set(data.var_names))

(1357593, 16906)


(812188, 16906)

In [14]:
data.obs

,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,Batch
09cbFwH3,2371,2371,689381.0,43521.0,6.313055,SRA275902
0AdkBIuA,2568,2568,601183.0,41842.0,6.959944,SRA275902
0frx9XJU,1570,1570,341170.0,33386.0,9.785737,SRA275902
0NaEBg2H,3141,3141,438603.0,48809.0,11.128287,SRA275902
0NQlCyEz,3289,3289,1020715.0,36329.0,3.559172,SRA275902
...,...,...,...,...,...,...
TTTTGTAATATA,794,794,1433.0,96.0,6.699232,SRA878024_SRS4660848
TTTTTCGCGCGG,609,609,988.0,71.0,7.186235,SRA878024_SRS4660848
TTTTTGGAGCCC,983,983,1930.0,102.0,5.284974,SRA878024_SRS4660848
TTTTTTAGCCCC,323,323,426.0,15.0,3.521127,SRA878024_SRS4660848


# Sort torch.distributed, CUDA


- pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --upgrade --force-reinstall
- pip3 install fsspec==2023.10.0


In [7]:
from torch import nn
from torch.optim import Adam
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist

from sklearn.model_selection import train_test_split

In [8]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA available:", torch.version.cuda) 
print(torch.distributed.is_available())

CUDA available: True
CUDA available: 11.8
True


In [23]:
import os

rank = 0
world_size = 1
is_master = rank == 0

os.environ['RANK'] = str(rank)
os.environ['WORLD_SIZE'] = str(world_size) 
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12345'

In [ ]:
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP

dist.init_process_group(backend='gloo', init_method='env://')
device = torch.device("cuda:0")
torch.cuda.set_device(device)

- Padding tokens are used to make all sequences in a batch the same length, ensuring consistency for model training. Ignored during the learning process.

In [25]:
SEED = 2021
EPOCHS = 3
BATCH_SIZE = 3
GRADIENT_ACCUMULATION = 60
LEARNING_RATE = 1e-4
SEQ_LEN = 16906 + 1
VALIDATE_EVERY = 1
CLASS = 5 + 2
MASK_PROB = 0.15
REPLACE_PROB = 0.9
RANDOM_TOKEN_PROB = 0.0
MASK_TOKEN_ID = CLASS - 1  # AK: Problem or not?  
PAD_TOKEN_ID  = CLASS - 1
MASK_IGNORE_TOKEN_IDS = [0]
POS_EMBED_USING = True

model_name = 'panglao_pretrain'
ckpt_dir = './ckpts/'

In [12]:
class SCDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data

    def __getitem__(self, index):
        rand_start = random.randint(0, self.data.shape[0]-1)
        full_seq = self.data[rand_start].toarray()[0]        # Flattening from 2D to 1D. Takes one whole row. 
        full_seq[full_seq > (CLASS - 2)] = CLASS - 2         # Clip all values in full_seq to be at most CLASS - 2. Normalization. 
        full_seq = torch.from_numpy(full_seq).long()
        full_seq = torch.cat((full_seq, torch.tensor([0]))).to(device)
        return full_seq

    def __len__(self):
        return self.data.shape[0]

data_train, data_val = train_test_split(data.X, test_size=0.05,random_state=SEED)

train_dataset = SCDataset(data_train)
val_dataset = SCDataset(data_val)

In [26]:
train_dataset.__len__(), val_dataset.__len__()

(1289713, 67880)

In [27]:
train_sampler = DistributedSampler(train_dataset)
val_sampler   = utils.SequentialDistributedSampler(val_dataset, batch_size=BATCH_SIZE, world_size=world_size)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, sampler=val_sampler)

In [28]:
model = PerformerLM(num_tokens = CLASS, dim = 200, depth = 6, max_seq_len = SEQ_LEN,heads = 10,local_attn_heads = 0, g2v_position_emb = POS_EMBED_USING)
model.to(device)
model = DDP(model, device_ids=[local_rank], output_device=local_rank) # DistributedDataParallel

FileNotFoundError: [Errno 2] No such file or directory: '../data/gene2vec_16906.npy'